In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import geopandas as gpd
import pandas as pd
from pathlib import Path

from geowrangler.datasets import ookla
import geowrangler.area_zonal_stats as azs

/home/abbymoreno/miniconda3/envs/lacuna-fund2/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [3]:
# custom module
import sys

sys.path.append("../../../")  # include parent directory
from src.nightlights import *

# Extract Nightlights

### Set-up inputs and parameters

In [4]:
ADMIN_FPATH = Path("../../../data/01-admin-bounds/")
OUTPUT_DIR = Path("../../../data/04-output/")
NTL_DIR = Path("../../../data/02-raw/nightlights/")

### Load AOI

In [5]:
# Load the administrative boundaries for the 12 cities
aoi = gpd.read_file(ADMIN_FPATH / "renamed_target_admin_bounds.gpkg")
aoi.head(2)

,region_name,region_code,province_name,province_code,city_name,city_code,barangay_name,barangay_psgc_code,geometry
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053..."
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((120.33380 16.03974, 120.33389 16.039..."


In [6]:
# Log-in using EOG credentials
username = os.environ.get("EOG_USER", None)
username = username if username is not None else input("Username?")
password = os.environ.get("EOG_PASSWORD", None)
password = password if password is not None else getpass.getpass("Password?")

# set save_token to True so that access token gets stored in ~/.eog_creds/eog_access_token
access_token = get_eog_access_token(username, password, save_token=True)

2023-08-01 10:26:17.677 | INFO     | src.nightlights:get_eog_access_token:42 - Saving access_token to /home/abbymoreno/.eog_creds/eog_access_token.txt
2023-08-01 10:26:17.679 | INFO     | src.nightlights:get_eog_access_token:50 - Adding access token to environment var EOG_ACCESS_TOKEN


In [7]:
def download_nightlights(aoi, year="2021"):
    nightlights_aoi = generate_nightlights_feature(aoi, year)
    nightlights_df = nightlights_aoi.drop(columns=["geometry"])
    nightlights_df = pd.DataFrame(nightlights_df)
    nightlights_df.to_csv(OUTPUT_DIR / f"nightlights_{year}.csv")

In [8]:
download_nightlights(aoi, "2015")

2023-08-01 10:26:21.598 | INFO     | src.nightlights:generate_clipped_raster:376 - Using viirs global file as source raster: /home/abbymoreno/.geowrangler/nightlights/global/VNL_v21_npp_2015_global_vcmslcfg_c202205302300.average.dat.tif
2023-08-01 10:26:21.599 | INFO     | src.nightlights:download_url:161 - Using access token from environment var EOG_ACCESS_TOKEN


2023-08-01 10:26:22.456 | INFO     | src.nightlights:urlretrieve:104 - Retrieving https://eogdata.mines.edu/nighttime_light/annual/v21/2015/VNL_v21_npp_2015_global_vcmslcfg_c202205302300.average.dat.tif.gz into /home/abbymoreno/.geowrangler/nightlights/global/VNL_v21_npp_2015_global_vcmslcfg_c202205302300.average.dat.tif.gz


2023-08-01 11:13:18.824 | INFO     | src.nightlights:unzip_eog_gzip:224 - Unzipping /home/abbymoreno/.geowrangler/nightlights/global/VNL_v21_npp_2015_global_vcmslcfg_c202205302300.average.dat.tif.gz into /home/abbymoreno/.geowrangler/nightlights/global/VNL_v21_npp_2015_global_vcmslcfg_c202205302300.average.dat.tif
2023-08-01 11:15:32.237 | INFO     | src.nightlights:unzip_eog_gzip:235 - Deleting /home/abbymoreno/.geowrangler/nightlights/global/VNL_v21_npp_2015_global_vcmslcfg_c202205302300.average.dat.tif.gz
2023-08-01 11:15:33.231 | INFO     | src.nightlights:clip_raster:248 - Generating clipped raster file from /home/abbymoreno/.geowrangler/nightlights/global/VNL_v21_npp_2015_global_vcmslcfg_c202205302300.average.dat.tif to /home/abbymoreno/.geowrangler/nightlights/clip/37dc60633e9ed5141380b9681b5dabe4.tif with bounds [120.30772392   6.85810433 125.68101804  16.11340094] and buffer 0.1
2023-08-01 11:15:34.042 | INFO     | src.nightlights:generate_clipped_metadata:421 - Adding metadat

In [9]:
download_nightlights(aoi, "2014")

2023-08-01 11:25:06.627 | INFO     | src.nightlights:generate_clipped_raster:376 - Using viirs global file as source raster: /home/abbymoreno/.geowrangler/nightlights/global/VNL_v21_npp_2014_global_vcmslcfg_c202205302300.average.dat.tif
2023-08-01 11:25:06.630 | INFO     | src.nightlights:download_url:161 - Using access token from environment var EOG_ACCESS_TOKEN
2023-08-01 11:25:07.759 | INFO     | src.nightlights:urlretrieve:104 - Retrieving https://eogdata.mines.edu/nighttime_light/annual/v21/2014/VNL_v21_npp_2014_global_vcmslcfg_c202205302300.average.dat.tif.gz into /home/abbymoreno/.geowrangler/nightlights/global/VNL_v21_npp_2014_global_vcmslcfg_c202205302300.average.dat.tif.gz


2023-08-01 12:14:35.660 | INFO     | src.nightlights:unzip_eog_gzip:224 - Unzipping /home/abbymoreno/.geowrangler/nightlights/global/VNL_v21_npp_2014_global_vcmslcfg_c202205302300.average.dat.tif.gz into /home/abbymoreno/.geowrangler/nightlights/global/VNL_v21_npp_2014_global_vcmslcfg_c202205302300.average.dat.tif
2023-08-01 12:16:01.064 | INFO     | src.nightlights:unzip_eog_gzip:235 - Deleting /home/abbymoreno/.geowrangler/nightlights/global/VNL_v21_npp_2014_global_vcmslcfg_c202205302300.average.dat.tif.gz
2023-08-01 12:16:01.903 | INFO     | src.nightlights:clip_raster:248 - Generating clipped raster file from /home/abbymoreno/.geowrangler/nightlights/global/VNL_v21_npp_2014_global_vcmslcfg_c202205302300.average.dat.tif to /home/abbymoreno/.geowrangler/nightlights/clip/34528eec4e90f43c7687495950e2db88.tif with bounds [120.30772392   6.85810433 125.68101804  16.11340094] and buffer 0.1
2023-08-01 12:16:02.052 | INFO     | src.nightlights:generate_clipped_metadata:421 - Adding metadat

In [10]:
download_nightlights(aoi, "2013")

2023-08-01 12:54:54.467 | INFO     | src.nightlights:generate_clipped_raster:376 - Using viirs global file as source raster: /home/abbymoreno/.geowrangler/nightlights/global/VNL_v21_npp_2013_global_vcmcfg_c202205302300.average.dat.tif
2023-08-01 12:54:54.469 | INFO     | src.nightlights:download_url:161 - Using access token from environment var EOG_ACCESS_TOKEN
2023-08-01 12:54:55.580 | INFO     | src.nightlights:urlretrieve:104 - Retrieving https://eogdata.mines.edu/nighttime_light/annual/v21/2013/VNL_v21_npp_2013_global_vcmcfg_c202205302300.average.dat.tif.gz into /home/abbymoreno/.geowrangler/nightlights/global/VNL_v21_npp_2013_global_vcmcfg_c202205302300.average.dat.tif.gz


2023-08-01 13:25:55.775 | INFO     | src.nightlights:unzip_eog_gzip:224 - Unzipping /home/abbymoreno/.geowrangler/nightlights/global/VNL_v21_npp_2013_global_vcmcfg_c202205302300.average.dat.tif.gz into /home/abbymoreno/.geowrangler/nightlights/global/VNL_v21_npp_2013_global_vcmcfg_c202205302300.average.dat.tif
2023-08-01 13:27:03.271 | INFO     | src.nightlights:unzip_eog_gzip:235 - Deleting /home/abbymoreno/.geowrangler/nightlights/global/VNL_v21_npp_2013_global_vcmcfg_c202205302300.average.dat.tif.gz
2023-08-01 13:27:03.954 | INFO     | src.nightlights:clip_raster:248 - Generating clipped raster file from /home/abbymoreno/.geowrangler/nightlights/global/VNL_v21_npp_2013_global_vcmcfg_c202205302300.average.dat.tif to /home/abbymoreno/.geowrangler/nightlights/clip/5d031d0bb92414cad18e80045597389d.tif with bounds [120.30772392   6.85810433 125.68101804  16.11340094] and buffer 0.1
2023-08-01 13:27:04.069 | INFO     | src.nightlights:generate_clipped_metadata:421 - Adding metadata.json f

In [11]:
download_nightlights(aoi, "2012")

2023-08-01 13:32:17.757 | INFO     | src.nightlights:generate_clipped_raster:376 - Using viirs global file as source raster: /home/abbymoreno/.geowrangler/nightlights/global/VNL_v21_npp_201204-201303_global_vcmcfg_c202205302300.average.dat.tif
2023-08-01 13:32:17.758 | INFO     | src.nightlights:download_url:161 - Using access token from environment var EOG_ACCESS_TOKEN
2023-08-01 13:32:18.868 | INFO     | src.nightlights:urlretrieve:104 - Retrieving https://eogdata.mines.edu/nighttime_light/annual/v21/2012/VNL_v21_npp_201204-201303_global_vcmcfg_c202205302300.average.dat.tif.gz into /home/abbymoreno/.geowrangler/nightlights/global/VNL_v21_npp_201204-201303_global_vcmcfg_c202205302300.average.dat.tif.gz


2023-08-01 14:07:45.108 | INFO     | src.nightlights:unzip_eog_gzip:224 - Unzipping /home/abbymoreno/.geowrangler/nightlights/global/VNL_v21_npp_201204-201303_global_vcmcfg_c202205302300.average.dat.tif.gz into /home/abbymoreno/.geowrangler/nightlights/global/VNL_v21_npp_201204-201303_global_vcmcfg_c202205302300.average.dat.tif
2023-08-01 14:08:57.203 | INFO     | src.nightlights:unzip_eog_gzip:235 - Deleting /home/abbymoreno/.geowrangler/nightlights/global/VNL_v21_npp_201204-201303_global_vcmcfg_c202205302300.average.dat.tif.gz
2023-08-01 14:08:57.950 | INFO     | src.nightlights:clip_raster:248 - Generating clipped raster file from /home/abbymoreno/.geowrangler/nightlights/global/VNL_v21_npp_201204-201303_global_vcmcfg_c202205302300.average.dat.tif to /home/abbymoreno/.geowrangler/nightlights/clip/7482a117f900ee3af6fa27aa9f3e4368.tif with bounds [120.30772392   6.85810433 125.68101804  16.11340094] and buffer 0.1
2023-08-01 14:08:58.079 | INFO     | src.nightlights:generate_clipped_m